In [2]:
# %pip install langchain
# %pip install langchain-community
# %pip install pypdf
# %pip install unstructured
# %pip install openpyxl
# %pip install --upgrade --quiet  gpt4all > /dev/null
# %pip install --upgrade --quiet langchain-elasticsearch langchain-openai tiktoken langchain
# %pip install python-dotenv

## Loading Documents

In [43]:
import os
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredExcelLoader

In [97]:
directory = "KnowledgeBase"

allDocs = {}

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        print("Processing", filename)
        pdfLoader = PyPDFLoader(os.path.join(directory, filename))

        if "pdfs" not in allDocs:
            allDocs["pdfs"] = []

        allDocs["pdfs"].extend(pdfLoader.load())

    elif filename.endswith(".xlsx"):
        print("Processing", filename)
        excelLoader = UnstructuredExcelLoader(os.path.join(directory, filename))


        if "xlsx" not in allDocs:
            allDocs["xlsx"] = []

        allDocs["xlsx"].extend(excelLoader.load())
    else:
        continue

Processing Risikoanalyse.xlsx


/Users/I748655/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Processing A18_Liste_der_Dienstleister.xlsx
Processing Zuordnung_Prozesse_Standort.xlsx
Processing A21_Definition_Schutzbedarfskategorien.pdf
Processing Ergebnis_des_IT-Grundschutz-Checks.xlsx
Processing A05_Richtlinie_Lenkung_Korrektur_Vorbeugungsm.pdf
Processing Nicht_modellierte_Bausteine.xlsx
Processing Beschreibung_Recplast.pdf
Processing A3_Modellierung_Recplast_GmbH.xlsx
Processing A04_Richtlinie_internen_ISMS_Auditierung.pdf
Processing A11_Abgrenzung_des_Informationsverbunds.pdf
Processing A03_Richtlinie_Lenkung_Dokumenten_und_Aufzeichnungen.pdf
Processing A02_Richtlinie_Risikoanalyse.pdf
Processing A01_Sicherheitsleitlinie.pdf
Processing Risikoanalyse_GP_Ebene.xlsx
Processing A1_Strukturanalyse_RECPLAST_GmbH.xlsx


/Users/I748655/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Processing Schutzbedarfsfeststellung.xlsx
Processing Realisierungsplan.xlsx
Processing A1_Strukturanalyse-Abhaengigkeiten_RECPLAST_GmbH.xlsx


In [99]:
print("Loaded", len(allDocs["pdfs"]), "pdf pages")
print("Loaded", len(allDocs["xlsx"]), "excel sheets")

Loaded 129 pdf pages
Loaded 11 excel sheets


In [100]:
print("First excel page content: \n", allDocs["xlsx"][0].page_content[0:1000])

First excel page content: 
 


Zielobjekt
Gefährdung
Eintrittshäufigkeit ohne zusätzliche Maßnahmen
Auswirkung ohne zusätzliche Maßnahmen
Risiko ohne zusätzliche Maßnahmen
Risikobehandlungs-option
Erläuterung zur Risikobehandlung
Eintrittshäufigkeit mit zusätzlichen Maßnahmen
Auswirkung mit zusätzlichen Maßnahmen
Risiko mit zusätzlichen Maßnahmen


A002
G 0.11 Ausfall oder Störung von Dienstleistern
selten
vernachlässigbar
gering
Risikoakzeptanz
Aufgrund der bereits umgesetzten Maßnahmen wird das Risiko als vertretbar eingeschätzt.





A002
G 0.14 Ausspähen von Informationen (Spionage)
selten
begrenzt
gering
Risikoakzeptanz
Aufgrund der bereits umgesetzten Maßnahmen wird das Risiko als vertretbar eingeschätzt.





A002
G 0.15 Abhören
selten
beträchtlich
mittel
Risikoakzeptanz
Aufgrund der bereits umgesetzten Maßnahmen wird das Risiko als vertretbar eingeschätzt.





A002
G 0.16 Diebstahl von Geräten, Datenträgern oder Dokumenten
selten
begrenzt
gering
Risikoakzeptanz
Aufgrund der be

In [102]:
print("First pdf page content: \n", allDocs["pdfs"][0].page_content[0:1000])

First pdf page content: 
 Definition der 
Schutzbedarfskategorien
Seite 1 von 8



In [103]:
print("First excel page metadata:", allDocs["xlsx"][0].metadata)

First excel page metadata: {'source': 'KnowledgeBase/Risikoanalyse.xlsx'}


In [104]:
print("First pdf page metadata:", allDocs["pdfs"][78].metadata)

First pdf page metadata: {'source': 'KnowledgeBase/Beschreibung_Recplast.pdf', 'page': 63}


## Splitting Documents

In [105]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

r_splitter_excel = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\r\n", "\n", "\t", ",", " "]
)


In [107]:
splits={}

for key in allDocs:
    splits[key] = r_splitter.split_documents(allDocs[key])

In [108]:
for key in allDocs:
    print("Number of splits for", key, ":", len(splits[key]))

Number of splits for xlsx : 15917
Number of splits for pdfs : 1790


In [110]:
print("First 5 PDF splits: ", splits["pdfs"][:5])

First 5 splits from the first PDF document: [Document(page_content='Definition der \nSchutzbedarfskategorien\nSeite 1 von 8', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 0}), Document(page_content='Dokumenteneigenschaften\nVerantwortung Informationssicherheitsbeauftragter\nKlassifizierung S2 intern\nGültigkeitszeit Unbegrenzt', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 1}), Document(page_content='Überarbeitungsintervall Jährlich\nNächste Überarbeitung Oktober 2020\nDateiname A.2.1_Definition_Schutzbedarfskategorien\nDokumentenstatus und Freigabe', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 1}), Document(page_content='Status Version Datum Name und Abteilung/Firma\nErstellt 1.0 tt.mm.jjjj\nDokumentenhistorie\nVersion Änderung Datum Autor\n1.0 tt.mm.jjjj\nSeite 2 von 8', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page'

In [111]:
print("First 5 Excel splits", splits["xlsx"][:5])

First 5 Excel splits [Document(page_content='Zielobjekt\nGefährdung\nEintrittshäufigkeit ohne zusätzliche Maßnahmen\nAuswirkung ohne zusätzliche Maßnahmen\nRisiko ohne zusätzliche Maßnahmen', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'}), Document(page_content='Risikobehandlungs-option\nErläuterung zur Risikobehandlung\nEintrittshäufigkeit mit zusätzlichen Maßnahmen\nAuswirkung mit zusätzlichen Maßnahmen', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'}), Document(page_content='Risiko mit zusätzlichen Maßnahmen', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'}), Document(page_content='A002\nG 0.11 Ausfall oder Störung von Dienstleistern\nselten\nvernachlässigbar\ngering\nRisikoakzeptanz', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'}), Document(page_content='Aufgrund der bereits umgesetzten Maßnahmen wird das Risiko als vertretbar eingeschätzt.', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'})]


## Create Embeddings

In [6]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_elasticsearch import ElasticsearchStore
import numpy as np
from dotenv import load_dotenv
import os

In [9]:
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"

In [10]:
embedding = GPT4AllEmbeddings(model_name=model_name)

In [11]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [12]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [13]:
load_dotenv()  # This loads the .env file at the application start
password = os.getenv('passwd')
api_key = os.getenv('api_key')

In [14]:
cloud_id  = '802f868877384e9798b731802ffa4827:ZXVyb3BlLXdlc3QzLmdjcC5jbG91ZC5lcy5pbyQ0NzYyZTQ2YzQ5NDg0ODY5YTAzZDMxYzg5NjY2MjY3YyQ1ZjQ3NWI2NTQxOTI0NmZiODcxNDc3NjZlMTI4YWE2YQ=='
elastic_vector_search = ElasticsearchStore(
    es_cloud_id=cloud_id,
    index_name="embeddings_index",
    embedding=embedding,
    es_user="group13",
    es_password=password,
    es_api_key=api_key
)

In [147]:
# elastic_vector_search.add_documents(splits["pdfs"])

['8a67b8a9-6025-4097-8ad2-1f59a3aaedce',
 'c8212080-0520-4709-a80e-3927628469f0',
 '16c75b94-60c4-4ddb-aaeb-9d92ec67079a',
 '30164797-b59f-48f0-8815-0d72e354564b',
 '3779ac95-feaf-4273-93bf-fde511daba9a',
 '315d7b4e-3cde-435e-a584-ed6ac7bb2f88',
 'f2c3bada-07a9-4b39-973e-7bc3f5497e19',
 'dc6db5a1-364e-4629-9553-b2552667f1a3',
 'f086234e-19b4-4223-90df-06b4a94c0df9',
 '4eaf9ecd-12c8-4115-a8f9-1f5c07c4b98e',
 '9afc612f-65cf-4269-9fdd-f55bc0b5c537',
 '3afc53cf-6f13-4469-9e0e-184e52416744',
 'e68b3d24-16ac-48d8-9c69-c893565eb7ec',
 '5de4c803-1deb-418b-9cd0-19aee5d89f4f',
 '3fe542c8-b912-4ed8-8c99-333bde90aace',
 'cdfac090-5ad8-47db-83ad-91220174abca',
 'c1a31652-c57b-4ab9-85dd-1ac2cb0140b4',
 'a785fb45-a374-4bf2-a2c4-7e8c2638cb19',
 '11e83814-d42a-4f25-90e7-b87c5415e713',
 '92e443bc-13ad-41f5-8899-066c240aa50a',
 'ac353dc1-483f-46f9-a887-1568c710bc16',
 'df412c47-5c33-4fd6-98ab-eeb9fe1a2e29',
 '4722c804-e122-4d27-91e3-3d58b30f95ea',
 '9c62d8f2-eb47-48f0-99dc-f691c05be725',
 '2da8e934-3051-

In [148]:
# elastic_vector_search.add_documents(splits["xlsx"])

['b356681c-abd7-4452-acd0-f070e0ce19f5',
 'eb7f5318-2a61-4800-b2a2-40fc268d8e6c',
 '4be09428-f095-413a-b209-ecd207e17ca1',
 '4081718e-b79a-4459-aa34-0519a47ce601',
 '2a42ad7d-0c94-48a3-bf55-602339c3e327',
 'd78d5fb7-a267-48df-aa0d-8cf747de167b',
 'bbf091d0-550b-44ce-8b0c-e4b7b42da189',
 '990c54e6-7811-4c99-a4f1-9ec2bebe6c6d',
 '5c35a2ea-6b43-4e00-95c7-c8d443f5d47f',
 'a45d5e31-06ac-41d5-9fff-fb0b4a52245c',
 '2ba05bc0-161e-4cee-afc3-0fa675b4a258',
 '54375abb-8821-4e3d-aa64-93902e76f575',
 'd7560ae3-6718-4e3c-b438-3c4ff3d0bccd',
 '82e547aa-0232-441b-bb9c-c7077ef67982',
 '3bdb015b-02db-45c1-88cc-b2996b518226',
 'b5e62efb-2631-417b-81fa-b48e4df68928',
 '1380b061-8896-4d85-ae05-4a1d88dd388e',
 'b29a4590-8202-4a89-a064-96b969570dfd',
 '6bf47ad0-cec4-41b4-8420-58683d7937d7',
 '56162c3f-a2ed-4bff-8605-25bf41774730',
 '1fe955e6-53e8-4a34-85cf-e18bc330567a',
 'd381448a-0590-4b5e-b655-3c050634d8d0',
 'f688e42f-1dc0-4893-8e87-6cee9d80c3de',
 'ba3d5633-1951-4a07-8ccd-3e352f21360f',
 '9610adc1-e5e2-

## Testing

In [15]:
question = "Who is responsible for conducting the risk analysis and what methodology is used"

results = elastic_vector_search.similarity_search(question, 20)

In [16]:
results

[Document(page_content='Die Auditoren wählen für die Prüfung geeignete Methoden aus. Dies sind zum Beispiel Dokumentsichtung oder Interviews mit Anwendern, Administratoren', metadata={'source': 'KnowledgeBase/Ergebnis_des_IT-Grundschutz-Checks.xlsx'}),
 Document(page_content='Konventionalstrafen.\nSelbstbestimmungsrecht Beeinträchtigungen des informationellen Selbstbestimmungsrechts', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 4}),
 Document(page_content='Konventionalstrafen.\nSelbstbestimmungsrecht Beeinträchtigungen des informationellen Selbstbestimmungsrechts', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 4}),
 Document(page_content='Das Compliance Management ist Bestandteil des Audits für Informationssicherheit.\nRichtlinie zur internen ISMS-Auditierung', metadata={'source': 'KnowledgeBase/Ergebnis_des_IT-Grundschutz-Checks.xlsx'}),
 Document(page_content='Sowohl das Management als auch alle Mit